In [ ]:
%load_ext autoreload
import numpy as np
import seaborn as sns
import os
import ClassificationModelsimple
import geopandas as gpd
import tensorflow as tf


In [ ]:
#prepare data
df=gpd.read_file('Data/NepalEqUSGS.shp')
constcols=['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGV_Usgs']
Xdata=df[constcols].to_numpy()
Ydata=df.Landslide.to_numpy()

In [ ]:
#normalize Data
max=Xdata.max(axis=0)
min=Xdata.min(axis=0)
norm_xdata=(Xdata-min)/(max-min)

In [ ]:
#define model
clfmdl=ClassificationModelsimple.LandslideModel()
clfmdl.getclassificationModel(in_num=8, out_num=1)
clfmdl.getOptimizer()
clfmdl.compileModel()
#print(clfmdl.model.summary())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(norm_xdata, Ydata, test_size=0.30, random_state=42)

In [ ]:
def trainmodel(model,xdata,ydata):
    
    NUMBER_EPOCHS = 100
    filepath='checkpointsUSGSv2'
    BATCH_SIZE=32
    
    model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor="val_auc",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        options=None
    )
    print(type(xdata),type(ydata))
    hist = model.fit(x=xdata,
                     y=ydata,
                     epochs=NUMBER_EPOCHS,
                     batch_size=BATCH_SIZE,
                     validation_split=0.2,#auto validate using 20% of random samples at each epoch
                     verbose=1, callbacks=[model_checkpoint_callback],class_weight = {0: 1, 1: 5}

                    )
    return hist


In [ ]:
trainmodel(clfmdl.model,np.array(X_train,dtype=np.float32),np.expand_dims(np.array(y_train,dtype=np.float32),axis=-1))

In [ ]:
#predict in test set
preds=clfmdl.model.predict(X_test)

In [ ]:
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
fpr,tpr,thresholds=sklearn.metrics.roc_curve(y_test, preds)


In [ ]:
print(sklearn.metrics.auc(fpr,tpr))
print(sklearn.metrics.confusion_matrix(y_test,np.rint(preds)))

In [ ]:
preds2=preds
preds2[preds>0.80]=1
preds2[preds<=0.80]=0
sklearn.metrics.accuracy_score(y_test,preds2)

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % 0.8577,
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Landslide Classification")
plt.text(0.61, 0.15,"Accuracy=0.8131")
plt.legend(loc="lower right")
plt.savefig('Exaiplots/roc.pdf')
plt.show()


In [ ]:
#confusion  map
confusiondata=np.empty(Ydata.shape,dtype=object)
confusiondata[np.bitwise_and(Ydata==1,np.rint(preds2.flatten())==1)]='True Positive'
confusiondata[np.bitwise_and(Ydata==0,np.rint(preds2.flatten())==1)]='False Positive'
confusiondata[np.bitwise_and(Ydata==1,np.rint(preds2.flatten())==0)]='False Negative'
confusiondata[np.bitwise_and(Ydata==0,np.rint(preds2.flatten())==0)]='True Negative'

In [ ]:
df['confusion']=confusiondata

In [ ]:
import contextily as cx
df_wm = df.to_crs(epsg=3857)
ax=df_wm.plot(column='confusion',legend=True,figsize=(10, 10), alpha=0.7)
cx.add_basemap(ax,source=cx.providers.Esri.WorldShadedRelief)

In [ ]:
ax.get_figure().savefig('confusionmap2.pdf',facecolor=ax.get_facecolor())
